# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vallenar,-28.5708,-70.7581,294.25,39,100,1.82,CL,1691589045
1,1,maceio,-9.6658,-35.7353,298.84,73,75,5.14,BR,1691589046
2,2,oppdal,62.5943,9.6912,285.91,70,98,6.81,NO,1691589046
3,3,isangel,-19.5500,169.2667,293.33,72,70,6.06,VU,1691589046
4,4,margaret river,-33.9500,115.0667,284.48,69,74,0.87,AU,1691588864


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

import folium

# Create a base map
m = folium.Map(location=[0, 0], zoom_start=2)

# Define thresholds for humidity levels
low_humidity_threshold = 10
medium_humidity_threshold = 50

# Iterate through the city_data_df DataFrame
for index, row in city_data_df.iterrows():
    # Get the latitude, longitude, and humidity
    lat = row['Lat']
    lng = row['Lng']
    humidity = row['Humidity']

    # Determine the color based on humidity
    if humidity > medium_humidity_threshold:
        color = 'red' # High humidity
    elif humidity > low_humidity_threshold:
        color = 'orange' # Medium humidity
    else:
        color = 'yellow' # Low humidity

    # Create a circle at each city's coordinates with size and color based on humidity
    folium.Circle(
        location=[lat, lng],
        radius=humidity*1000, 
        color=color,
        fill=True,
        fill_opacity=0.5
    ).add_to(m)

# Save the map to an HTML file
m.save("humidity_map.html")



In [4]:
from IPython.display import IFrame

# Display the saved HTML file within the notebook
IFrame("humidity_map.html", width=800, height=400)


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

# Define the ideal weather conditions
ideal_max_temp = (city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 300)
ideal_wind_speed = city_data_df['Wind Speed'] <= 10
ideal_cloudiness = city_data_df['Cloudiness'] >= 0

# Apply the conditions to filter the DataFrame
ideal_weather_df = city_data_df[ideal_max_temp & ideal_wind_speed & ideal_cloudiness]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vallenar,-28.5708,-70.7581,294.25,39,100,1.82,CL,1691589045
1,1,maceio,-9.6658,-35.7353,298.84,73,75,5.14,BR,1691589046
2,2,oppdal,62.5943,9.6912,285.91,70,98,6.81,NO,1691589046
3,3,isangel,-19.5500,169.2667,293.33,72,70,6.06,VU,1691589046
4,4,margaret river,-33.9500,115.0667,284.48,69,74,0.87,AU,1691588864


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Assuming city_data_df is the DataFrame containing the necessary information
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty "Hotel Name" column
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,vallenar,CL,-28.5708,-70.7581,39,
1,maceio,BR,-9.6658,-35.7353,73,
2,oppdal,NO,62.5943,9.6912,70,
3,isangel,VU,-19.5500,169.2667,72,
4,margaret river,AU,-33.9500,115.0667,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
radius = 10000  # approx 7mi - radius is in meters
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
vallenar - nearest hotel: Hotel del Marqués
maceio - nearest hotel: Maceió Hostel e Pousada
oppdal - nearest hotel: Quality Hotel Skifer
isangel - nearest hotel: Tanna Lodge
margaret river - nearest hotel: Margaret River Hotel
tiksi - nearest hotel: Арктика
adamstown - nearest hotel: No hotel found
mamuju - nearest hotel: Hotel d'Maleo
coyhaique - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
waitangi - nearest hotel: Hotel Chathams
ambodifotatra - nearest hotel: Les Palmiers
aasiaat - nearest hotel: SØMA
west island - nearest hotel: Cocos Village Bungalows
hermanus - nearest hotel: Aloe guest house
uturoa - nearest hotel: Hawaiki Nui hotel
port-aux-francais - nearest hotel: Keravel
hastings - nearest hotel: White Rock Hotel
taro - nearest hotel: Padma Resort Ubud
taroa - nearest hotel: No hotel found
puerto ayora - nearest h

pires do rio - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
hanceville - nearest hotel: No hotel found
garmisch-partenkirchen - nearest hotel: aja Hotel
hobyo - nearest hotel: No hotel found
revelstoke - nearest hotel: Grizz Hotel
bethanie - nearest hotel: No hotel found
xilin hot - nearest hotel: Hotel hailibu 120
jiangyou - nearest hotel: No hotel found
laascaanood - nearest hotel: Warsame Hotel mnesta
choyr - nearest hotel: Сансар Хайрхан зочид буудал
ternate - nearest hotel: Boulivard ternate
susuman - nearest hotel: No hotel found
elektrenai - nearest hotel: Perkūnkiemis
pagar alam - nearest hotel: Villa Dempo Flower
ogulin - nearest hotel: Hotel Frankopan
acapulco de juarez - nearest hotel: Hotel del Valle
kanash - nearest hotel: Нарспи
illizi - nearest hotel: بوناقة
quellon - nearest hotel: Aliwen
gaigeturi - nearest hotel: 제주 토비스콘도
qoryooley - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
rexburg - nearest hotel: Quality Inn
ust-nera 

malango - nearest hotel: Madarana
cookeville - nearest hotel: Best Western
itaituba - nearest hotel: TG Hotel
colonia - nearest hotel: Wasserturm Hotel Cologne
nyurba - nearest hotel: Марина
bowen - nearest hotel: Whitsunday Sands Resort
yanchep - nearest hotel: Yanchep Inn
lagunas - nearest hotel: Hospedaje Eco
kovrov - nearest hotel: Визит
praia da vitoria - nearest hotel: Salles
northumberland - nearest hotel: Newcastle House Hotel
nova sintra - nearest hotel: Residência Ka Dencho
mitu - nearest hotel: Hotel Brasilia Vaupés
conde - nearest hotel: Paladino
gagliano del capo - nearest hotel: Masseria Palane
dali - nearest hotel: 风花雪月大酒店
urangan - nearest hotel: Ramada
el sauzal de rodriguez - nearest hotel: Hotel Sausalito
pala - nearest hotel: No hotel found
kirtland - nearest hotel: No hotel found
fort st. john - nearest hotel: Fort St. John Motor Inn
worthington - nearest hotel: Magnuson Grand Columbus North Hotel
mina clavero - nearest hotel: Hotel Palace
mendoza - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vallenar,CL,-28.5708,-70.7581,39,Hotel del Marqués
1,maceio,BR,-9.6658,-35.7353,73,Maceió Hostel e Pousada
2,oppdal,NO,62.5943,9.6912,70,Quality Hotel Skifer
3,isangel,VU,-19.5500,169.2667,72,Tanna Lodge
4,margaret river,AU,-33.9500,115.0667,69,Margaret River Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# %%capture --no-display

import folium

latitude = hotel_df['Lat'].mean()
longitude = hotel_df['Lng'].mean()
zoom_level = 4

# map centered around desired latitude and longitude
m = folium.Map(location=[latitude, longitude], zoom_start=zoom_level)

# Loop through the DataFrame to add points for each city
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    hotel_name = row['Hotel Name']
    country = row['Country']
    
    # Create a tooltip with the hotel name and country
    tooltip = f"Hotel: {hotel_name}<br>Country: {country}"

    # Add the point to the map with the tooltip
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        tooltip=tooltip,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

# Save the map or display it
m.save('hotel_map.html')



# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [12]:
IFrame("hotel_map.html", width=800, height=400)